In [1]:
#importing required libraries
import cv2
import numpy as np
from sklearn.decomposition import PCA
from google.colab import drive
import os

In [2]:
#mounting the google drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
# Load video
video = cv2.VideoCapture('/content/drive/My Drive/SEM 4/CMPE 295B Final Year Project/vp9_compressed_videos_Gaming_1080P-0ce6_cbr.webm')

In [4]:
# Get video properties
fps = int(video.get(cv2.CAP_PROP_FPS))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
input_size = os.path.getsize('/content/drive/My Drive/SEM 4/CMPE 295B Final Year Project/vp9_compressed_videos_Gaming_1080P-0ce6_cbr.webm')

In [5]:
print(fps, width, height, num_frames, input_size)

30 1920 1080 599 8220384


In [6]:
# Set new video resolution
new_width = 960
new_height = 540

# Create VideoWriter object to write new video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('/content/drive/My Drive/SEM 4/CMPE 295B Final Year Project/new_video.mp4', fourcc, fps, (new_width, new_height))

# Loop over video frames
while True:
    # Read frame
    ret, frame = video.read()
    if not ret: break

    # Resize frame
    resized_frame = cv2.resize(frame, (new_width, new_height))

    # Write resized frame to new video
    out.write(resized_frame)

# Release resources
video.release()
out.release()

In [7]:
# load the new video
video = cv2.VideoCapture('/content/drive/My Drive/SEM 4/CMPE 295B Final Year Project/new_video.mp4')

In [8]:
# Get video properties
fps = int(video.get(cv2.CAP_PROP_FPS))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
input_size = os.path.getsize('/content/drive/My Drive/SEM 4/CMPE 295B Final Year Project/new_video.mp4')

In [9]:
print(fps, width, height, num_frames, input_size)

30 960 540 599 34613544


In [10]:
# Loop over video frames
flattened_frames = []
for i in range(num_frames):
    # Read frame
    ret, frame = video.read()
    if not ret: break

    # Convert frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Flatten frame into a 1D array
    flattened_frame = gray_frame.flatten()

    # Append compressed frame to output list
    flattened_frames.append(flattened_frame)

In [11]:
print(len(flattened_frames), flattened_frames[0].shape)

599 (518400,)


In [12]:
pca = PCA(n_components = 240)

In [13]:
flattened_frames = np.array(flattened_frames)
flattened_frames.shape

(599, 518400)

In [14]:
# Get size of each compressed frame
compressed_frame_size = flattened_frames[0].nbytes
total_compressed_size = compressed_frame_size * 599

# Print compressed size
print(f'Total compressed size: {total_compressed_size} bytes')

Total compressed size: 310521600 bytes


In [15]:
flattened_frames = flattened_frames.T
flattened_frames.shape

(518400, 599)

In [16]:
flattened_frames[0].shape

(599,)

In [17]:
flattened_frames = pca.fit_transform(flattened_frames)

In [18]:
flattened_frames.shape

(518400, 240)

In [19]:
# Get size of each compressed frame
compressed_frame_size = flattened_frames.T[0].nbytes
total_compressed_size = compressed_frame_size * 240

# Print compressed size
print(f'Total compressed size: {total_compressed_size} bytes')

Total compressed size: 995328000 bytes


In [20]:
reconstructed_frames = pca.inverse_transform(flattened_frames)

In [21]:
reconstructed_frames.shape

(518400, 599)

In [22]:
reconstructed_frames = reconstructed_frames.T
reconstructed_frames.shape

(599, 518400)

In [23]:
reconstructed_frames[0]

array([98.38693712, 65.23388342, 39.28029883, ..., 90.71788597,
       41.46189217, 40.56258719])

In [24]:
reconstructed_frames_fixed = []
for reconstructed_frame in reconstructed_frames:
    
    # Reshape frame into original shape
    reconstructed_frame = reconstructed_frame.reshape((height, width))

    # converting to float32
    reconstructed_frame_float32 = np.float32(reconstructed_frame)
    
    # Convert frame to RGB
    rgb_frame = cv2.cvtColor(reconstructed_frame_float32, cv2.COLOR_GRAY2BGR)

    # Append reconstructed frame to output list
    reconstructed_frames_fixed.append(reconstructed_frame)

In [25]:
print(len(reconstructed_frames_fixed), reconstructed_frames_fixed[0].shape)

599 (540, 960)


In [26]:
# Save reconstructed video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('/content/drive/My Drive/SEM 4/CMPE 295B Final Year Project/reconstructed_video.mp4', fourcc, fps, (width, height))
for frame in reconstructed_frames_fixed:
    converted_frame = cv2.convertScaleAbs(frame)
    out.write(converted_frame)
out.release()